In [1]:
import datetime 

import numpy as np
import pandas as pd

from sklearn.experimental import enable_iterative_imputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import IterativeImputer

# Dataset Overview

The dataset is composed of three csv files.

Features contains several features valid for each (store, date) tuple from 2010 to 2013

Sales contains the weekly sales value (our prediction target) for each (store, department, date) tuple

Stores contains information about each store in the dataset.

In [2]:
features_df = pd.read_csv('data/features.csv')
sales_df = pd.read_csv('data/sales.csv')
stores_df = pd.read_csv('data/stores.csv')

In [3]:
features_df.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,05/02/2010,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,12/02/2010,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,19/02/2010,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,26/02/2010,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,05/03/2010,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [4]:
sales_df.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,05/02/2010,24924.50,False
1,1,1,12/02/2010,46039.49,True
2,1,1,19/02/2010,41595.55,False
3,1,1,26/02/2010,19403.54,False
4,1,1,05/03/2010,21827.90,False


In [5]:
stores_df.head()

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


Type is a categorical feature with only three possible options. So it's better to one hot encode this feature for an easier modeling process.

In [6]:
one_hot_encoding = pd.get_dummies(stores_df['Type'])
stores_df = stores_df.join(one_hot_encoding)
stores_df = stores_df.drop(columns=['Type'])
stores_df.head()

,Store,Size,A,B,C
0,1,151315,1,0,0
1,2,202307,1,0,0
2,3,37392,0,1,0
3,4,205863,1,0,0
4,5,34875,0,1,0


All three dataframes are grouped into a single one.

In [7]:
week_sales_df = sales_df.groupby(['Date', 'Store', 'Dept']).agg({'Weekly_Sales': 'sum'}).sort_index()
week_sales_df.reset_index(inplace=True)
training_df = pd.merge(features_df, week_sales_df, how='left', on=["Date", "Store"])
training_df = pd.merge(training_df, stores_df, how='left', on=["Store"])

In [8]:
training_df.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday,Dept,Weekly_Sales,Size,A,B,C
0,1,05/02/2010,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False,1.0,24924.50,151315,1,0,0
1,1,05/02/2010,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False,2.0,50605.27,151315,1,0,0
2,1,05/02/2010,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False,3.0,13740.12,151315,1,0,0
3,1,05/02/2010,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False,4.0,39954.04,151315,1,0,0
4,1,05/02/2010,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False,5.0,32229.38,151315,1,0,0


The dataframe contains several NaN values that must be removed/replaced

In [9]:
training_df.isna().sum()

Store                0
Date                 0
Temperature          0
Fuel_Price           0
MarkDown1       270892
MarkDown2       310793
MarkDown3       284667
MarkDown4       286859
MarkDown5       270138
CPI                585
Unemployment       585
IsHoliday            0
Dept              1755
Weekly_Sales      1755
Size                 0
A                    0
B                    0
C                    0
dtype: int64

In order to interpolate NaN values, the dataframe must be indexed by DateTime object

In [10]:
def gen_datetime(date_str):
    return datetime.datetime.strptime(date_str, '%d/%m/%Y')

training_df['DateTime'] = training_df['Date'].map(gen_datetime)

In [11]:
training_df = training_df.set_index(['DateTime', 'Store', 'Dept']).sort_index()

In [12]:
training_df.head()

Date  Temperature  Fuel_Price  MarkDown1  \
DateTime   Store Dept                                                   
2010-02-05 1     1.0   05/02/2010        42.31       2.572        NaN   
                 2.0   05/02/2010        42.31       2.572        NaN   
                 3.0   05/02/2010        42.31       2.572        NaN   
                 4.0   05/02/2010        42.31       2.572        NaN   
                 5.0   05/02/2010        42.31       2.572        NaN   

                       MarkDown2  MarkDown3  MarkDown4  MarkDown5         CPI  \
DateTime   Store Dept                                                           
2010-02-05 1     1.0         NaN        NaN        NaN        NaN  211.096358   
                 2.0         NaN        NaN        NaN        NaN  211.096358   
                 3.0         NaN        NaN        NaN        NaN  211.096358   
                 4.0         NaN        NaN        NaN        NaN  211.096358   
                 5.0         NaN        NaN        NaN        NaN  211.096358   

                       Unemployment  IsHoliday  Weekly_Sales    Size  A  B  C  
DateTime   Store Dept                                                          
2010-02-05 1     1.0          8.106      False      24924.50  151315  1  0  0  
                 2.0          8.106      False      50605.27  151315  1  0  0  
                 3.0          8.106      False      13740.12  151315  1  0  0  
                 4.0          8.106      False      39954.04  151315  1  0  0  
                 5.0          8.106      False      32229.38  151315  1  0  0

In [13]:
training_df['CPI'] = training_df['CPI'].interpolate(method='linear')
training_df['Unemployment'] = training_df['Unemployment'].interpolate(method='linear')

In [14]:
training_df.isna().sum()

Date                 0
Temperature          0
Fuel_Price           0
MarkDown1       270892
MarkDown2       310793
MarkDown3       284667
MarkDown4       286859
MarkDown5       270138
CPI                  0
Unemployment         0
IsHoliday            0
Weekly_Sales      1755
Size                 0
A                    0
B                    0
C                    0
dtype: int64

In [15]:
training_df['MarkDown1'] = IterativeImputer(random_state=0).fit_transform(training_df['MarkDown1'].values.reshape(-1, 1))
training_df['MarkDown2'] = IterativeImputer(random_state=0).fit_transform(training_df['MarkDown2'].values.reshape(-1, 1))
training_df['MarkDown3'] = IterativeImputer(random_state=0).fit_transform(training_df['MarkDown3'].values.reshape(-1, 1))
training_df['MarkDown4'] = IterativeImputer(random_state=0).fit_transform(training_df['MarkDown4'].values.reshape(-1, 1))
training_df['MarkDown5'] = IterativeImputer(random_state=0).fit_transform(training_df['MarkDown5'].values.reshape(-1, 1))

In [16]:
training_df = training_df.reset_index()

In [17]:
training_df = training_df[training_df['Dept'].notna()]

In [18]:
training_df.isna().sum()

DateTime        0
Store           0
Dept            0
Date            0
Temperature     0
Fuel_Price      0
MarkDown1       0
MarkDown2       0
MarkDown3       0
MarkDown4       0
MarkDown5       0
CPI             0
Unemployment    0
IsHoliday       0
Weekly_Sales    0
Size            0
A               0
B               0
C               0
dtype: int64

In [19]:
training_df.head()

,DateTime,Store,Dept,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday,Weekly_Sales,Size,A,B,C
0,2010-02-05,1,1.0,05/02/2010,42.31,2.572,7246.604247,3337.597289,1449.098932,3382.019834,4618.743783,211.096358,8.106,False,24924.50,151315,1,0,0
1,2010-02-05,1,2.0,05/02/2010,42.31,2.572,7246.604247,3337.597289,1449.098932,3382.019834,4618.743783,211.096358,8.106,False,50605.27,151315,1,0,0
2,2010-02-05,1,3.0,05/02/2010,42.31,2.572,7246.604247,3337.597289,1449.098932,3382.019834,4618.743783,211.096358,8.106,False,13740.12,151315,1,0,0
3,2010-02-05,1,4.0,05/02/2010,42.31,2.572,7246.604247,3337.597289,1449.098932,3382.019834,4618.743783,211.096358,8.106,False,39954.04,151315,1,0,0
4,2010-02-05,1,5.0,05/02/2010,42.31,2.572,7246.604247,3337.597289,1449.098932,3382.019834,4618.743783,211.096358,8.106,False,32229.38,151315,1,0,0


Date should be decomposed into Year and a cyclical feature for Day of Year

In [20]:
training_df['Year'] = training_df['Date'].map(lambda x: int(x[-4:]))

In [21]:
training_df.head()

,DateTime,Store,Dept,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday,Weekly_Sales,Size,A,B,C,Year
0,2010-02-05,1,1.0,05/02/2010,42.31,2.572,7246.604247,3337.597289,1449.098932,3382.019834,4618.743783,211.096358,8.106,False,24924.50,151315,1,0,0,2010
1,2010-02-05,1,2.0,05/02/2010,42.31,2.572,7246.604247,3337.597289,1449.098932,3382.019834,4618.743783,211.096358,8.106,False,50605.27,151315,1,0,0,2010
2,2010-02-05,1,3.0,05/02/2010,42.31,2.572,7246.604247,3337.597289,1449.098932,3382.019834,4618.743783,211.096358,8.106,False,13740.12,151315,1,0,0,2010
3,2010-02-05,1,4.0,05/02/2010,42.31,2.572,7246.604247,3337.597289,1449.098932,3382.019834,4618.743783,211.096358,8.106,False,39954.04,151315,1,0,0,2010
4,2010-02-05,1,5.0,05/02/2010,42.31,2.572,7246.604247,3337.597289,1449.098932,3382.019834,4618.743783,211.096358,8.106,False,32229.38,151315,1,0,0,2010


In [22]:
def day_of_year(date_str):
    date = datetime.datetime.strptime(date_str, '%d/%m/%Y')
    return date.timetuple().tm_yday

training_df['DayOfYear'] = training_df['Date'].map(day_of_year)
training_df['DayOfYearCos'] = np.cos(training_df['DayOfYear'])
training_df['DayOfYearSin'] = np.sin(training_df['DayOfYear'])

In [23]:
training_df.head()

,DateTime,Store,Dept,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,...,IsHoliday,Weekly_Sales,Size,A,B,C,Year,DayOfYear,DayOfYearCos,DayOfYearSin
0,2010-02-05,1,1.0,05/02/2010,42.31,2.572,7246.604247,3337.597289,1449.098932,3382.019834,...,False,24924.50,151315,1,0,0,2010,36,-0.127964,-0.991779
1,2010-02-05,1,2.0,05/02/2010,42.31,2.572,7246.604247,3337.597289,1449.098932,3382.019834,...,False,50605.27,151315,1,0,0,2010,36,-0.127964,-0.991779
2,2010-02-05,1,3.0,05/02/2010,42.31,2.572,7246.604247,3337.597289,1449.098932,3382.019834,...,False,13740.12,151315,1,0,0,2010,36,-0.127964,-0.991779
3,2010-02-05,1,4.0,05/02/2010,42.31,2.572,7246.604247,3337.597289,1449.098932,3382.019834,...,False,39954.04,151315,1,0,0,2010,36,-0.127964,-0.991779
4,2010-02-05,1,5.0,05/02/2010,42.31,2.572,7246.604247,3337.597289,1449.098932,3382.019834,...,False,32229.38,151315,1,0,0,2010,36,-0.127964,-0.991779


In [24]:
training_df = training_df.drop(columns=['Date'])

In [25]:
training_df.head()

,DateTime,Store,Dept,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,...,IsHoliday,Weekly_Sales,Size,A,B,C,Year,DayOfYear,DayOfYearCos,DayOfYearSin
0,2010-02-05,1,1.0,42.31,2.572,7246.604247,3337.597289,1449.098932,3382.019834,4618.743783,...,False,24924.50,151315,1,0,0,2010,36,-0.127964,-0.991779
1,2010-02-05,1,2.0,42.31,2.572,7246.604247,3337.597289,1449.098932,3382.019834,4618.743783,...,False,50605.27,151315,1,0,0,2010,36,-0.127964,-0.991779
2,2010-02-05,1,3.0,42.31,2.572,7246.604247,3337.597289,1449.098932,3382.019834,4618.743783,...,False,13740.12,151315,1,0,0,2010,36,-0.127964,-0.991779
3,2010-02-05,1,4.0,42.31,2.572,7246.604247,3337.597289,1449.098932,3382.019834,4618.743783,...,False,39954.04,151315,1,0,0,2010,36,-0.127964,-0.991779
4,2010-02-05,1,5.0,42.31,2.572,7246.604247,3337.597289,1449.098932,3382.019834,4618.743783,...,False,32229.38,151315,1,0,0,2010,36,-0.127964,-0.991779


Normalizing values between 0 and 1

In [26]:
training_df['Store'] = MinMaxScaler().fit_transform(training_df['Store'].values.reshape((-1, 1)))
training_df['Temperature'] = MinMaxScaler().fit_transform(training_df['Temperature'].values.reshape((-1, 1)))
training_df['Fuel_Price'] = MinMaxScaler().fit_transform(training_df['Fuel_Price'].values.reshape((-1, 1)))
training_df['MarkDown1'] = MinMaxScaler().fit_transform(training_df['MarkDown1'].values.reshape((-1, 1)))
training_df['MarkDown2'] = MinMaxScaler().fit_transform(training_df['MarkDown2'].values.reshape((-1, 1)))
training_df['MarkDown3'] = MinMaxScaler().fit_transform(training_df['MarkDown3'].values.reshape((-1, 1)))
training_df['MarkDown4'] = MinMaxScaler().fit_transform(training_df['MarkDown4'].values.reshape((-1, 1)))
training_df['MarkDown5'] = MinMaxScaler().fit_transform(training_df['MarkDown5'].values.reshape((-1, 1)))
training_df['CPI'] = MinMaxScaler().fit_transform(training_df['CPI'].values.reshape((-1, 1)))
training_df['Unemployment'] = MinMaxScaler().fit_transform(training_df['Unemployment'].values.reshape((-1, 1)))
training_df['IsHoliday'] = training_df['IsHoliday'].astype(int)
training_df['Dept'] = MinMaxScaler().fit_transform(training_df['Dept'].values.reshape((-1, 1)))
training_df['Weekly_Sales'] = MinMaxScaler().fit_transform(training_df['Weekly_Sales'].values.reshape((-1, 1)))
training_df['Size'] = MinMaxScaler().fit_transform(training_df['Size'].values.reshape((-1, 1)))

In [27]:
training_df.head()

,DateTime,Store,Dept,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,...,IsHoliday,Weekly_Sales,Size,A,B,C,Year,DayOfYear,DayOfYearCos,DayOfYearSin
0,2010-02-05,0.0,0.000000,0.434149,0.0501,0.081744,0.034388,0.010435,0.05012,0.041368,...,0,0.042851,0.630267,1,0,0,2010,36,-0.127964,-0.991779
1,2010-02-05,0.0,0.010204,0.434149,0.0501,0.081744,0.034388,0.010435,0.05012,0.041368,...,0,0.079638,0.630267,1,0,0,2010,36,-0.127964,-0.991779
2,2010-02-05,0.0,0.020408,0.434149,0.0501,0.081744,0.034388,0.010435,0.05012,0.041368,...,0,0.026829,0.630267,1,0,0,2010,36,-0.127964,-0.991779
3,2010-02-05,0.0,0.030612,0.434149,0.0501,0.081744,0.034388,0.010435,0.05012,0.041368,...,0,0.064380,0.630267,1,0,0,2010,36,-0.127964,-0.991779
4,2010-02-05,0.0,0.040816,0.434149,0.0501,0.081744,0.034388,0.010435,0.05012,0.041368,...,0,0.053315,0.630267,1,0,0,2010,36,-0.127964,-0.991779


In [28]:
training_df.tail()

,DateTime,Store,Dept,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,...,IsHoliday,Weekly_Sales,Size,A,B,C,Year,DayOfYear,DayOfYearCos,DayOfYearSin
421565,2012-10-26,1.0,0.938776,0.595988,0.706413,0.045333,0.003091,0.000911,0.003138,0.006672,...,0,0.010710,0.451136,0,1,0,2012,300,-0.022097,-0.999756
421566,2012-10-26,1.0,0.948980,0.595988,0.706413,0.045333,0.003091,0.000911,0.003138,0.006672,...,0,0.014600,0.451136,0,1,0,2012,300,-0.022097,-0.999756
421567,2012-10-26,1.0,0.959184,0.595988,0.706413,0.045333,0.003091,0.000911,0.003138,0.006672,...,0,0.087391,0.451136,0,1,0,2012,300,-0.022097,-0.999756
421568,2012-10-26,1.0,0.979592,0.595988,0.706413,0.045333,0.003091,0.000911,0.003138,0.006672,...,0,0.016913,0.451136,0,1,0,2012,300,-0.022097,-0.999756
421569,2012-10-26,1.0,0.989796,0.595988,0.706413,0.045333,0.003091,0.000911,0.003138,0.006672,...,0,0.008689,0.451136,0,1,0,2012,300,-0.022097,-0.999756


Split training set from 2010 to 2011 and prediction set from 2012 to 2013

In [29]:
training_df = training_df.set_index('DateTime')

In [30]:
training_set_df = training_df[:'2012-01-01']

In [31]:
training_set_df.head()

,Store,Dept,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,...,IsHoliday,Weekly_Sales,Size,A,B,C,Year,DayOfYear,DayOfYearCos,DayOfYearSin
DateTime,,,,,,,,,,,,,,,,,,,,,
2010-02-05,0.0,0.000000,0.434149,0.0501,0.081744,0.034388,0.010435,0.05012,0.041368,0.8405,...,0,0.042851,0.630267,1,0,0,2010,36,-0.127964,-0.991779
2010-02-05,0.0,0.010204,0.434149,0.0501,0.081744,0.034388,0.010435,0.05012,0.041368,0.8405,...,0,0.079638,0.630267,1,0,0,2010,36,-0.127964,-0.991779
2010-02-05,0.0,0.020408,0.434149,0.0501,0.081744,0.034388,0.010435,0.05012,0.041368,0.8405,...,0,0.026829,0.630267,1,0,0,2010,36,-0.127964,-0.991779
2010-02-05,0.0,0.030612,0.434149,0.0501,0.081744,0.034388,0.010435,0.05012,0.041368,0.8405,...,0,0.064380,0.630267,1,0,0,2010,36,-0.127964,-0.991779
2010-02-05,0.0,0.040816,0.434149,0.0501,0.081744,0.034388,0.010435,0.05012,0.041368,0.8405,...,0,0.053315,0.630267,1,0,0,2010,36,-0.127964,-0.991779


In [32]:
training_set_df.tail()

,Store,Dept,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,...,IsHoliday,Weekly_Sales,Size,A,B,C,Year,DayOfYear,DayOfYearCos,DayOfYearSin
DateTime,,,,,,,,,,,,,,,,,,,,,
2011-12-30,1.0,0.938776,0.389922,0.459419,0.040657,0.421885,0.002235,0.005294,0.005673,0.622702,...,1,0.009190,0.451136,0,1,0,2011,364,0.911143,-0.412091
2011-12-30,1.0,0.948980,0.389922,0.459419,0.040657,0.421885,0.002235,0.005294,0.005673,0.622702,...,1,0.011414,0.451136,0,1,0,2011,364,0.911143,-0.412091
2011-12-30,1.0,0.959184,0.389922,0.459419,0.040657,0.421885,0.002235,0.005294,0.005673,0.622702,...,1,0.067432,0.451136,0,1,0,2011,364,0.911143,-0.412091
2011-12-30,1.0,0.979592,0.389922,0.459419,0.040657,0.421885,0.002235,0.005294,0.005673,0.622702,...,1,0.015125,0.451136,0,1,0,2011,364,0.911143,-0.412091
2011-12-30,1.0,0.989796,0.389922,0.459419,0.040657,0.421885,0.002235,0.005294,0.005673,0.622702,...,1,0.007939,0.451136,0,1,0,2011,364,0.911143,-0.412091


In [33]:
test_set_df = training_df['2012-01-01':]

In [34]:
test_set_df.head()

,Store,Dept,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,...,IsHoliday,Weekly_Sales,Size,A,B,C,Year,DayOfYear,DayOfYearCos,DayOfYearSin
DateTime,,,,,,,,,,,,,,,,,,,,,
2012-01-06,0.0,0.000000,0.499706,0.343186,0.070811,0.210706,0.001216,0.021488,0.077021,0.925683,...,0,0.030880,0.630267,1,0,0,2012,6,0.96017,-0.279415
2012-01-06,0.0,0.010204,0.499706,0.343186,0.070811,0.210706,0.001216,0.021488,0.077021,0.925683,...,0,0.070865,0.630267,1,0,0,2012,6,0.96017,-0.279415
2012-01-06,0.0,0.020408,0.499706,0.343186,0.070811,0.210706,0.001216,0.021488,0.077021,0.925683,...,0,0.027095,0.630267,1,0,0,2012,6,0.96017,-0.279415
2012-01-06,0.0,0.030612,0.499706,0.343186,0.070811,0.210706,0.001216,0.021488,0.077021,0.925683,...,0,0.065772,0.630267,1,0,0,2012,6,0.96017,-0.279415
2012-01-06,0.0,0.040816,0.499706,0.343186,0.070811,0.210706,0.001216,0.021488,0.077021,0.925683,...,0,0.040245,0.630267,1,0,0,2012,6,0.96017,-0.279415


In [35]:
test_set_df.tail()

,Store,Dept,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,...,IsHoliday,Weekly_Sales,Size,A,B,C,Year,DayOfYear,DayOfYearCos,DayOfYearSin
DateTime,,,,,,,,,,,,,,,,,,,,,
2012-10-26,1.0,0.938776,0.595988,0.706413,0.045333,0.003091,0.000911,0.003138,0.006672,0.654796,...,0,0.010710,0.451136,0,1,0,2012,300,-0.022097,-0.999756
2012-10-26,1.0,0.948980,0.595988,0.706413,0.045333,0.003091,0.000911,0.003138,0.006672,0.654796,...,0,0.014600,0.451136,0,1,0,2012,300,-0.022097,-0.999756
2012-10-26,1.0,0.959184,0.595988,0.706413,0.045333,0.003091,0.000911,0.003138,0.006672,0.654796,...,0,0.087391,0.451136,0,1,0,2012,300,-0.022097,-0.999756
2012-10-26,1.0,0.979592,0.595988,0.706413,0.045333,0.003091,0.000911,0.003138,0.006672,0.654796,...,0,0.016913,0.451136,0,1,0,2012,300,-0.022097,-0.999756
2012-10-26,1.0,0.989796,0.595988,0.706413,0.045333,0.003091,0.000911,0.003138,0.006672,0.654796,...,0,0.008689,0.451136,0,1,0,2012,300,-0.022097,-0.999756


In [36]:
X_training = training_set_df.drop(columns=['Weekly_Sales', 'Year', 'DayOfYear']).values
y_training = training_set_df['Weekly_Sales'].values
X_prediction = test_set_df.drop(columns=['Weekly_Sales', 'Year', 'DayOfYear']).values
y_prediction = test_set_df['Weekly_Sales'].values

In [37]:
X_training.shape, y_training.shape, X_prediction.shape, y_prediction.shape

((294132, 18), (294132,), (127438, 18), (127438,))

Split training set into train and test set

In [38]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_training, y_training, test_size=0.33, random_state=0)

In [39]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((197068, 18), (197068,), (97064, 18), (97064,))

Decision Tree is able to predict value of unseen years with score of 0.85

In [40]:
from sklearn.tree import DecisionTreeRegressor

dtr_regressor = DecisionTreeRegressor(max_depth=50, min_samples_leaf=200, random_state=0)

In [41]:
dtr_regressor = dtr_regressor.fit(X_train, y_train)
dtr_regressor.score(X_test, y_test)

0.8339429299658346

In [42]:
future_pred = dtr_regressor.predict(X_prediction)
dtr_regressor.score(X_prediction, y_prediction)

0.8507626393467799

Random Forest is able to predict value of unseen years with score of 0.86

In [43]:
from sklearn.ensemble import RandomForestRegressor

rfr_regressor = RandomForestRegressor(n_estimators=20, max_depth=100, min_samples_leaf=150, random_state=0)

In [44]:
rfr_regressor = rfr_regressor.fit(X_train, y_train)
rfr_regressor.score(X_test, y_test)

0.839565073492495

In [45]:
future_pred = rfr_regressor.predict(X_prediction)
rfr_regressor.score(X_prediction, y_prediction)

0.8615569217734452

Gradient Boosting Regressor is able to predict the value of unseen years with score of 0.87

In [46]:
from sklearn.ensemble import GradientBoostingRegressor

gbr_regressor = GradientBoostingRegressor(n_estimators=20, max_depth=150, min_samples_leaf=150, random_state=0)

In [47]:
gbr_regressor = gbr_regressor.fit(X_train, y_train)
gbr_regressor.score(X_test, y_test)

0.8833556278576009

In [48]:
future_pred = gbr_regressor.predict(X_prediction)
gbr_regressor.score(X_prediction, y_prediction)

0.8690338706793365